### Links To CDC for Raw Data

## Covid-19 Infection 
### url-1: https://data.cdc.gov/Case-Surveillance/Weekly-United-States-COVID-19-Cases-and-Deaths-by-/pwn4-m3yp/about_data
<!-- ## url-1: https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36 -->

## Covid-19 Vaccine Distribution
### url-2: https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-Jurisdi/unsk-b7fc

In [8]:
!pip install psycopg2



    ERROR: Command errored out with exit status 1:


  Using cached psycopg2-2.9.8.tar.gz (383 kB)



     command: 'C:\Users\tahir\anaconda3\envs\PythonAdv\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\tahir\\AppData\\Local\\Temp\\pip-install-qdw17n2b\\psycopg2\\setup.py'"'"'; __file__='"'"'C:\\Users\\tahir\\AppData\\Local\\Temp\\pip-install-qdw17n2b\\psycopg2\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\tahir\AppData\Local\Temp\pip-pip-egg-info-zvhqou0n'
         cwd: C:\Users\tahir\AppData\Local\Temp\pip-install-qdw17n2b\psycopg2\
    Complete output (23 lines):
    running egg_info
    creating C:\Users\tahir\AppData\Local\Temp\pip-pip-egg-info-zvhqou0n\psycopg2.egg-info
    writing C:\Users\tahir\AppData\Local\Temp\pip-pip-egg-info-zvhqou0n\psycopg2.egg-info\PKG-INFO
    writing dependency_links to C:\Users\tahir\AppData\Local\Temp\pip-pip-egg-info-zvhqou0n\psycopg2.egg-info\dependency_li

In [1]:
# import dependencies
import numpy as np
import pandas as pd
import datetime as dt
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine, inspect, func
import psycopg2
import requests
from sodapy import Socrata

# import cdc key for API
from cdc_token import my_token

C:\Users\tahir\AppData\Local\Temp\ipykernel_14784\2334959182.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## USA Monthly Cases by State 

In [8]:
# set a client with a token
client = Socrata("data.cdc.gov", my_token)

# results returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
# results = client.get("9mfq-cb36", limit=10000000)
results = client.get("pwn4-m3yp", limit=1000000000)
# Convert to pandas DataFrame and display
covid_df = pd.DataFrame.from_records(results)
covid_df.tail()

,date_updated,state,start_date,end_date,tot_cases,new_cases,tot_deaths,new_deaths,new_historic_cases,new_historic_deaths
10375,2023-04-13T00:00:00.000,WY,2023-04-06T00:00:00.000,2023-04-12T00:00:00.000,186284.0,158.0,2023.0,3.0,0,0
10376,2023-04-20T00:00:00.000,WY,2023-04-13T00:00:00.000,2023-04-19T00:00:00.000,186399.0,115.0,2027.0,4.0,0,0
10377,2023-04-27T00:00:00.000,WY,2023-04-20T00:00:00.000,2023-04-26T00:00:00.000,186539.0,140.0,2030.0,3.0,0,0
10378,2023-05-04T00:00:00.000,WY,2023-04-27T00:00:00.000,2023-05-03T00:00:00.000,186668.0,129.0,2031.0,1.0,0,0
10379,2023-02-02T00:00:00.000,NV,2023-01-26T00:00:00.000,2023-02-01T00:00:00.000,884178.0,1269.0,11867.0,18.0,0,-17


In [10]:
# add date columns which only has year and month for use with groupby later on.. 
covid_df['date'] = pd.to_datetime(covid_df['date_updated']).dt.to_period('M').astype(str)
covid_df.tail()

,date_updated,state,start_date,end_date,tot_cases,new_cases,tot_deaths,new_deaths,new_historic_cases,new_historic_deaths,date
10375,2023-04-13T00:00:00.000,WY,2023-04-06T00:00:00.000,2023-04-12T00:00:00.000,186284.0,158.0,2023.0,3.0,0,0,2023-04
10376,2023-04-20T00:00:00.000,WY,2023-04-13T00:00:00.000,2023-04-19T00:00:00.000,186399.0,115.0,2027.0,4.0,0,0,2023-04
10377,2023-04-27T00:00:00.000,WY,2023-04-20T00:00:00.000,2023-04-26T00:00:00.000,186539.0,140.0,2030.0,3.0,0,0,2023-04
10378,2023-05-04T00:00:00.000,WY,2023-04-27T00:00:00.000,2023-05-03T00:00:00.000,186668.0,129.0,2031.0,1.0,0,0,2023-05
10379,2023-02-02T00:00:00.000,NV,2023-01-26T00:00:00.000,2023-02-01T00:00:00.000,884178.0,1269.0,11867.0,18.0,0,-17,2023-02


In [11]:
covid_df.dtypes

date_updated           object
state                  object
start_date             object
end_date               object
tot_cases              object
new_cases              object
tot_deaths             object
new_deaths             object
new_historic_cases     object
new_historic_deaths    object
date                   object
dtype: object

In [13]:
# convert objects to float
covid_df["tot_cases"] =  covid_df.tot_cases.astype(float)
covid_df["tot_deaths"] = covid_df.tot_deaths.astype(float)

# select only the columns we gonna use and display df
covid_df = covid_df[["date", "state", "tot_cases", "tot_deaths"]]
covid_df.head()

,date,state,tot_cases,tot_deaths
0,2023-02,AZ,2434631.0,33042.0
1,2022-12,LA,1507707.0,18345.0
2,2023-02,GA,3061141.0,42324.0
3,2023-03,LA,1588259.0,18858.0
4,2023-02,LA,1548508.0,18572.0


In [6]:
# covid_test = covid_max.loc[(covid_max['state'] == 'TX')]
# covid_test.head(60)

In [12]:
# group by states to be used later
# state_df = covid_df.groupby('state').mean().reset_index()
# state_df.head()

In [14]:
# add total recovery columns to df
covid_df["Total_recovered"] = covid_df.tot_cases - covid_df.tot_deaths
covid_df.tail()

,date,state,tot_cases,tot_deaths,Total_recovered
10375,2023-04,WY,186284.0,2023.0,184261.0
10376,2023-04,WY,186399.0,2027.0,184372.0
10377,2023-04,WY,186539.0,2030.0,184509.0
10378,2023-05,WY,186668.0,2031.0,184637.0
10379,2023-02,NV,884178.0,11867.0,872311.0


In [16]:
# groupby date and state and take the mean value to get monthly average..
covid_monthly_df = round(covid_df.groupby(["date", "state"]).max().reset_index(), 2)
covid_monthly_df

,date,state,tot_cases,tot_deaths,Total_recovered
0,2020-01,AK,0.0,0.0,0.0
1,2020-01,AL,0.0,0.0,0.0
2,2020-01,AR,0.0,0.0,0.0
3,2020-01,AS,0.0,0.0,0.0
4,2020-01,AZ,1.0,0.0,1.0
...,...,...,...,...,...
2455,2023-05,VT,154243.0,967.0,153276.0
2456,2023-05,WA,1957759.0,16186.0,1941573.0
2457,2023-05,WI,2030717.0,16600.0,2014117.0
2458,2023-05,WV,650556.0,8125.0,642431.0


In [17]:
# rename columns. name gotta be compatable wth sqlite database naming potocol.
covid_monthly_df = covid_monthly_df.rename(columns={"date": "Date", "state": "State", "tot_cases": "Total_cases", "tot_deaths": "Total_death"})
covid_monthly_df.head()

,Date,State,Total_cases,Total_death,Total_recovered
0,2020-01,AK,0.0,0.0,0.0
1,2020-01,AL,0.0,0.0,0.0
2,2020-01,AR,0.0,0.0,0.0
3,2020-01,AS,0.0,0.0,0.0
4,2020-01,AZ,1.0,0.0,1.0


In [18]:
# add death and recovery percentage columns. change NA values to zero
covid_monthly_df['Death_percent'] = round((covid_monthly_df.Total_death/covid_monthly_df.Total_cases)*100, 2)
covid_monthly_df['Recovery_percent'] = round((covid_monthly_df.Total_recovered/covid_monthly_df.Total_cases)*100, 2)
covid_monthly_df=covid_monthly_df.fillna(0)
covid_infec_df = covid_monthly_df.copy()
covid_monthly_df.tail()

,Date,State,Total_cases,Total_death,Total_recovered,Death_percent,Recovery_percent
2455,2023-05,VT,154243.0,967.0,153276.0,0.63,99.37
2456,2023-05,WA,1957759.0,16186.0,1941573.0,0.83,99.17
2457,2023-05,WI,2030717.0,16600.0,2014117.0,0.82,99.18
2458,2023-05,WV,650556.0,8125.0,642431.0,1.25,98.75
2459,2023-05,WY,187034.0,2031.0,185003.0,1.09,98.91


In [19]:
# save df as a csv file
covid_monthly_df.to_csv('../Data/monthly.csv')

## USA Covid-19 Infection Map Data

In [20]:
# since data is aggregated using rolling average, we only goinog to keep last month data.
covid_infec_df = covid_infec_df.drop_duplicates(subset=['State'], keep='last', inplace=False, ignore_index=True)
covid_infec_df.tail()

,Date,State,Total_cases,Total_death,Total_recovered,Death_percent,Recovery_percent
55,2023-05,VT,154243.0,967.0,153276.0,0.63,99.37
56,2023-05,WA,1957759.0,16186.0,1941573.0,0.83,99.17
57,2023-05,WI,2030717.0,16600.0,2014117.0,0.82,99.18
58,2023-05,WV,650556.0,8125.0,642431.0,1.25,98.75
59,2023-05,WY,187034.0,2031.0,185003.0,1.09,98.91


## USA Covid-19 Vaccine Status by States

In [21]:
# set a client with a token
client = Socrata("data.cdc.gov", my_token)

# results returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("unsk-b7fc", limit=100000)

# Convert to pandas DataFrame and display
covid_vac_df = pd.DataFrame.from_records(results)
covid_vac_df.head()

,date,mmwr_week,location,distributed,distributed_janssen,distributed_moderna,distributed_pfizer,distributed_novavax,distributed_unk_manuf,dist_per_100k,...,dist_bivalent_mod,bivalent_booster_5plus,bivalent_booster_5plus_pop_pct,bivalent_booster_12plus,bivalent_booster_12plus_pop_pct,bivalent_booster_18plus,bivalent_booster_18plus_pop_pct,bivalent_booster_65plus,bivalent_booster_65plus_pop_pct,second_booster
0,2023-05-10T00:00:00.000,19,NE,5481710,152400,1647380,2905630,7400,0,283379,...,193420,340508,18.9,332054,20.5,319161,21.9,151146,48.4,NaN
1,2023-05-10T00:00:00.000,19,LA,10282120,330500,3807980,5164550,10100,0,221178,...,328400,359506,8.3,356199,9.1,349934,9.8,184844,24.9,NaN
2,2023-05-10T00:00:00.000,19,GA,28727475,869100,9763000,14773655,43400,0,270569,...,1023320,1126791,11.3,1103841,12.3,1067468,13.2,476297,31.4,NaN
3,2023-05-10T00:00:00.000,19,WY,1281755,49300,490040,585605,3700,0,221466,...,50600,65920,12.1,64919,13.2,63244,14.2,34832,35.1,NaN
4,2023-05-10T00:00:00.000,19,CO,17769135,501900,5402640,9029715,43600,0,308560,...,757520,1268501,23.4,1229981,24.9,1176686,26.2,462990,55,NaN


In [22]:
covid_vac_df.dtypes

date                               object
mmwr_week                          object
location                           object
distributed                        object
distributed_janssen                object
                                    ...  
bivalent_booster_18plus            object
bivalent_booster_18plus_pop_pct    object
bivalent_booster_65plus            object
bivalent_booster_65plus_pop_pct    object
second_booster                     object
Length: 109, dtype: object

In [23]:
# add date columns which only has year and month for use with groupby later on.. 
covid_vac_df['Date'] = pd.to_datetime(covid_vac_df['date']).dt.to_period('M').astype(str)
covid_vac_df.tail()

,date,mmwr_week,location,distributed,distributed_janssen,distributed_moderna,distributed_pfizer,distributed_novavax,distributed_unk_manuf,dist_per_100k,...,bivalent_booster_5plus,bivalent_booster_5plus_pop_pct,bivalent_booster_12plus,bivalent_booster_12plus_pop_pct,bivalent_booster_18plus,bivalent_booster_18plus_pop_pct,bivalent_booster_65plus,bivalent_booster_65plus_pop_pct,second_booster,Date
38483,2020-12-13T00:00:00.000,51,AS,3900,0,0,0,NaN,0,7003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-12
38484,2020-12-13T00:00:00.000,51,VI,975,0,0,0,NaN,0,931,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-12
38485,2020-12-13T00:00:00.000,51,MP,4875,0,0,0,NaN,0,8570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-12
38486,2020-12-13T00:00:00.000,51,US,13650,0,0,0,NaN,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-12
38487,2020-12-13T00:00:00.000,51,GU,3900,0,0,0,NaN,0,2353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-12


In [24]:
# convert objects to float
covid_vac_df["distributed"] =  covid_vac_df.distributed.astype(float)
covid_vac_df["dist_per_100k"] = covid_vac_df.dist_per_100k.astype(float)
covid_vac_df["administered"] =  covid_vac_df.administered.astype(float)
covid_vac_df["admin_per_100k"] = covid_vac_df.admin_per_100k.astype(float)

# select only the columns we gonna use and display df
covid_vac_df = covid_vac_df[["Date", "location", "distributed", "dist_per_100k", "administered", "admin_per_100k"]]
covid_vac_df.head()

,Date,location,distributed,dist_per_100k,administered,admin_per_100k
0,2023-05,NE,5481710.0,283379.0,3822190.0,197590.0
1,2023-05,LA,10282120.0,221178.0,6961453.0,149748.0
2,2023-05,GA,28727475.0,270569.0,17124791.0,161290.0
3,2023-05,WY,1281755.0,221466.0,854132.0,147580.0
4,2023-05,CO,17769135.0,308560.0,13033446.0,226325.0


In [25]:
# rename columns. name gotta be compatable wth sqlite database naming potocol.
covid_vac_df = covid_vac_df.rename(columns={"location": "State", "distributed": "Distributed", "dist_per_100k": "Dist_per_100k", "administered": "Administered", "admin_per_100k": "Admin_per_100k"})
covid_vac_df.tail()

,Date,State,Distributed,Dist_per_100k,Administered,Admin_per_100k
38483,2020-12,AS,3900.0,7003.0,0.0,0.0
38484,2020-12,VI,975.0,931.0,0.0,0.0
38485,2020-12,MP,4875.0,8570.0,0.0,0.0
38486,2020-12,US,13650.0,4.0,0.0,0.0
38487,2020-12,GU,3900.0,2353.0,0.0,0.0


In [26]:
# since data is aggregated using rolling average, we only goinog to keep month Data.
covid_vac_df = covid_vac_df.drop_duplicates(subset=['State'], keep='first', inplace=False, ignore_index=True)
covid_vac_df.head()

,Date,State,Distributed,Dist_per_100k,Administered,Admin_per_100k
0,2023-05,NE,5481710.0,283379.0,3822190.0,197590.0
1,2023-05,LA,10282120.0,221178.0,6961453.0,149748.0
2,2023-05,GA,28727475.0,270569.0,17124791.0,161290.0
3,2023-05,WY,1281755.0,221466.0,854132.0,147580.0
4,2023-05,CO,17769135.0,308560.0,13033446.0,226325.0


In [27]:
covid_vac_df.dtypes

Date               object
State              object
Distributed       float64
Dist_per_100k     float64
Administered      float64
Admin_per_100k    float64
dtype: object

In [28]:
# save df as a csv file
covid_vac_df.to_csv('../Data/vaccine.csv')

## USA Total Cases and Death by State

In [21]:
# upload data file and read it ito a pandas dataframe
# file = "../Data/US_COVID-19_Deaths.csv"
# covid_stats_df = pd.read_csv(file)
# covid_stats_df.head()

In [22]:
# select relevant columns only
# covid_stats_df = covid_stats_df[['State/Territory', 'Total Cases', 'Case Rate per 100000', 'Total Deaths', 'Death Rate per 100000']]
# covid_stats_df.head()

In [23]:
# add a state abbreviation column
# covid_stats_df['State'] = state_df['state']
# covid_stats_df.head()

In [24]:
# rename columns and drop any column with NA
# covid_stats_df = covid_stats_df.rename(columns={"state": "State", "Total Cases": "Total_cases", "Case Rate per 100000": "Case_rate_per_100k", "Total Deaths": "Total_deaths", "Death Rate per 100000": "Death_rate_per_100k"})
# covid_stats_df = covid_stats_df[["State", "Total_cases", "Case_rate_per_100k", "Total_deaths", "Death_rate_per_100k"]].fillna(0)
# covid_stats_df.tail()

## USA COVID-19 data with latitude and longtitude to create interactive map

In [25]:
# upload file and read to pandas dataframe
# file = "../Data/US_COVID-19_stats.csv"
# us_covid_df = pd.read_csv(file)
# us_covid_df.head()

In [26]:
# select relevant columns
# us_covid_df = us_covid_df[["Province_State", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Mortality_Rate"]]
# us_covid_df.head()

In [27]:
# add state column to be filled with state abbreviation
# us_covid_df["State"] = "" 
# us_covid_df.head()

In [28]:
# # upload States and their abbreviation
# us_state_abbrev = {
    
#     'Alabama': 'AL',
#     'Alaska': 'AK',
#     'American Samoa': 'AS',
#     'Arizona': 'AZ',
#     'Arkansas': 'AR',
#     'California': 'CA',
#     'Colorado': 'CO',
#     'Connecticut': 'CT',
#     'Delaware': 'DE',
#     'Diamond Princess': 'DP',
#     'District of Columbia': 'DC',
#     'Florida': 'FL',
#     'Georgia': 'GA',
#     'Grand Princess': 'GP',
#     'Guam': 'GU',
#     'Hawaii': 'HI',
#     'Idaho': 'ID',
#     'Illinois': 'IL',
#     'Indiana': 'IN',
#     'Iowa': 'IA',
#     'Kansas': 'KS',
#     'Kentucky': 'KY',
#     'Louisiana': 'LA',
#     'Maine': 'ME',
#     'Maryland': 'MD',
#     'Massachusetts': 'MA',
#     'Michigan': 'MI',
#     'Minnesota': 'MN',
#     'Mississippi': 'MS',
#     'Missouri': 'MO',
#     'Montana': 'MT',
#     'Nebraska': 'NE',
#     'Nevada': 'NV',
#     'New Hampshire': 'NH',
#     'New Jersey': 'NJ',
#     'New Mexico': 'NM',
#     'New York': 'NY',
#     'North Carolina': 'NC',
#     'North Dakota': 'ND',
#     'Northern Mariana Islands':'MP',
#     'Ohio': 'OH',
#     'Oklahoma': 'OK',
#     'Oregon': 'OR',
#     'Pennsylvania': 'PA',
#     'Puerto Rico': 'PR',
#     'Rhode Island': 'RI',
#     'South Carolina': 'SC',
#     'South Dakota': 'SD',
#     'Tennessee': 'TN',
#     'Texas': 'TX',
#     'Utah': 'UT',
#     'Vermont': 'VT',
#     'Virgin Islands': 'VI',
#     'Virginia': 'VA',
#     'Washington': 'WA',
#     'West Virginia': 'WV',
#     'Wisconsin': 'WI',
#     'Wyoming': 'WY'
# }
# abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [29]:
# loop through and assign abbreviation acoordinely
# for index, row in us_covid_df.iterrows():
#     us_covid_df.loc[index, "State"] = us_state_abbrev[row[0]]

# us_covid_df.head()

In [ ]:
# loop through and assign abbreviation acoordinely
# for index, row in us_covid_df.iterrows():
#     if row["State"] == "DP":
#         us_covid_df.loc[index, "Lat"] = 14.5214
#         us_covid_df.loc[index, "Long_"] = 120.9709

#     elif row["State"] == "GP":
#         us_covid_df.loc[index, "Lat"] = 32.1584
#         us_covid_df.loc[index, "Long_"] = 117.5676

    # if row["State"] == FSM:
    #     us_covid_df.loc[index, "Lat"] = 6.8874
    #     us_covid_df.loc[index, "Long"] = 158.2150

    # elif row["State"] == NYC:
    #     us_covid_df.loc[index, "Lat"] = 40.7128
    #     us_covid_df.loc[index, "Long"] = -74.0060

    # elif row["State"] == PW:
    #     us_covid_df.loc[index, "Lat"] = 7.5150
    #     us_covid_df.loc[index, "Long"] = 134.5825

    # elif row["State"] == RMI:
    #     us_covid_df.loc[index, "Lat"] = 7.1315
    #     us_covid_df.loc[index, "Long"] = 171.1845

# us_covid_df.head()

In [30]:
# rename columns and add recovery percent column
# us_covid_df = us_covid_df.rename(columns={"Long_": "Long", "Mortality_Rate":"Death_percent"})
# us_covid_df["Recovery_percent"]=us_covid_df.Recovered/us_covid_df.Confirmed
# us_covid_df = us_covid_df[["State", "Lat", "Long", "Death_percent", "Recovery_percent"]].fillna(0)
# us_covid_df.head()

In [32]:
# loop through and assign abbreviation acoordinely
# for index, row in us_covid_df.iterrows():
#     if row["State"] == "DP":
#         us_covid_df.loc[index, "Lat"] = 14.5214
#         us_covid_df.loc[index, "Long"] = 120.9709

#     elif row["State"] == "GP":
#         us_covid_df.loc[index, "Lat"] = 32.1584
#         us_covid_df.loc[index, "Long"] = 117.5676

    # elif row["State"] == FSM:
    #     us_covid_df.loc[index, "Lat"] = 6.8874
    #     us_covid_df.loc[index, "Long"] = 158.2150

    # elif row["State"] == NYC:
    #     us_covid_df.loc[index, "Lat"] = 40.7128
    #     us_covid_df.loc[index, "Long"] = -74.0060

    # elif row["State"] == PW:
    #     us_covid_df.loc[index, "Lat"] = 7.5150
    #     us_covid_df.loc[index, "Long"] = 134.5825

    # elif row["State"] == RMI:
    #     us_covid_df.loc[index, "Lat"] = 7.1315
    #     us_covid_df.loc[index, "Long"] = 171.1845

# us_covid_df.head()

In [33]:
# merge two df into one dataframe using left joint.
# us_covid_combine_df = pd.merge(covid_stats_df, us_covid_df, how='left', on=['State','State'])
# us_covid_combine_df.head()

In [34]:
# save to a csv file
# us_covid_combine_df.to_csv('../Data/states.csv')

## Add Geo property to states in old state file

In [35]:
# Add Lat and Long Columns
# geomap_df = covid_monthly_df.groupby('State').mean().reset_index()
# covid_monthly_df["Long"] = ""
# geomap_df.head()

## World COVID-19 Data by Country

In [36]:
# upload file and read into a pandas dataframe
# file = "../Data/World_COVID-19_Stats.csv"
# world_covid_df = pd.read_csv(file)
# world_covid_df.head()

In [37]:
# select relevant columns only
# world_covid_df = world_covid_df[["Country_Region", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Case-Fatality_Ratio"]]
# world_covid_df.head()

In [38]:
# rename columns and add a recovery percent column
# world_covid_df = world_covid_df.rename(columns={"Long_": "Long", "Case-Fatality_Ratio":"Death_percent", "Country_Region": "Country"})
# world_covid_df["Recovery_percent"]=world_covid_df.Recovered/world_covid_df.Confirmed
# world_covid_df.head()

In [39]:
# add missing geoinfo to country
# for index, row in world_covid_df.iterrows(): 
#     if row["Country"] == "Canada":
#         world_covid_df.loc[index, "Lat"] = 56.1304
#         world_covid_df.loc[index, "Long"] = 106.3468
# world_covid_df

In [40]:
# save file as a csv
# world_covid_df.to_csv('../Data/world.csv')

## build SQlite DataBase

In [29]:
# import modules
import sqlite3
from sqlite3 import Error

# series of functions to  build database tables in sqlite database.

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return conn
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def drop_table(conn, drop_table_sql):
    """ drop a table from the drop_table_sql statement
    :param conn: Connection object
    :param drop_table_sql: a drop TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(drop_table_sql)
    except Error as e:
        print(e)

# build empty tables with correct columns names
def main():		
    database = r"../DataBase/covid-19.db"
    sql_create_monthly_table = """CREATE TABLE monthly(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT, 
                                        Date text NOT NULL,
                                        State text,
                                        Total_cases float,
                                        Total_death float,
                                        Total_recovered float,
                                        Death_percent float,
                                        Recovery_percent float
                                    ); """

    sql_drop_monthly_table = """DROP TABLE if exists monthly;"""

    sql_create_infection_table = """CREATE TABLE infection(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT, 
                                        Date text NOT NULL,
                                        State text,
                                        Total_cases float,
                                        Total_death float,
                                        Total_recovered float,
                                        Death_percent float,
                                        Recovery_percent float
                                    ); """

    sql_drop_infection_table = """DROP TABLE if exists infection;"""

    sql_create_vaccine_table = """CREATE TABLE vaccine(
                                        id INTEGER PRIMARY KEY AUTOINCREMENT, 
                                        Date text NOT NULL,
                                        State text,
                                        Distributed float,
                                        Dist_per_100k float,
                                        Administered float,
                                        Admin_per_100k float
                                    ); """

    sql_drop_vaccine_table = """DROP TABLE if exists vaccine;"""
    
    # sql_create_states_table = """CREATE TABLE states(
    #                                     id INTEGER PRIMARY KEY AUTOINCREMENT,
    #                                     State text,
    #                                     Total_cases float,
    #                                     Case_rate_per_100k float,
    #                                     Total_deaths float,
    #                                     Death_rate_per_100k float,
    #                                     Lat float,
    #                                     Long float,
    #                                     Death_percent float,
    #                                     Recovery_percent float
    #                                 );"""

    # sql_drop_states_table = """DROP TABLE if exists states;"""    

    # sql_create_world_table = """CREATE TABLE world(
    #                                     id INTEGER PRIMARY KEY AUTOINCREMENT,
    #                                     Country text,
    #                                     Lat float,
    #                                     Long float,
    #                                     Confirmed float,
    #                                     Deaths float,
    #                                     Recovered float,
    #                                     Recovery_percent float,
    #                                     Death_percent float
    #                                 );"""

    # sql_drop_world_table = """DROP TABLE if exists world;"""

    # create a database connection
    conn = create_connection(database)
    
    # create tables
    if conn is not None:
        # create covid_monthly table
        drop_table(conn, sql_drop_monthly_table)
        create_table(conn, sql_create_monthly_table)

        # create covid_infection table
        drop_table(conn, sql_drop_infection_table)
        create_table(conn, sql_create_infection_table)

        # create covid_vaccine table
        drop_table(conn, sql_drop_vaccine_table)
        create_table(conn, sql_create_vaccine_table)

        # create covid_states table
        # drop_table(conn, sql_drop_states_table)
        # create_table(conn, sql_create_states_table)

        #create covid_world table
        # drop_table(conn, sql_drop_world_table)
        # create_table(conn, sql_create_world_table)
        
    # error handling
    else:
        print("Error! cannot create the database connection.")
if __name__ == '__main__':
    main()

In [30]:
# create engine connection to database
engine = create_engine('sqlite:///../DataBase/covid-19.db')

# # populate tables with information from dataframes
covid_monthly_df.to_sql(name='monthly', con=engine, if_exists = "append", index=False)
covid_infec_df.to_sql(name='infection', con=engine, if_exists = "append", index=False)
covid_vac_df.to_sql(name='vaccine', con=engine, if_exists = "append", index=False)
# world_covid_df.to_sql(name='world', con=engine, if_exists = "append", index= False)
# us_covid_combine_df.to_sql(name='states', con=engine, if_exists = "append", index=False)

66

In [31]:
# querry monthly data from database. to test and make sure tables are working
pd.read_sql('select * from monthly', engine)

,id,Date,State,Total_cases,Total_death,Total_recovered,Death_percent,Recovery_percent
0,1,2020-01,AK,0.0,0.0,0.0,0.00,0.00
1,2,2020-01,AL,0.0,0.0,0.0,0.00,0.00
2,3,2020-01,AR,0.0,0.0,0.0,0.00,0.00
3,4,2020-01,AS,0.0,0.0,0.0,0.00,0.00
4,5,2020-01,AZ,1.0,0.0,1.0,0.00,100.00
...,...,...,...,...,...,...,...,...
2455,2456,2023-05,VT,154243.0,967.0,153276.0,0.63,99.37
2456,2457,2023-05,WA,1957759.0,16186.0,1941573.0,0.83,99.17
2457,2458,2023-05,WI,2030717.0,16600.0,2014117.0,0.82,99.18
2458,2459,2023-05,WV,650556.0,8125.0,642431.0,1.25,98.75


In [32]:
pd.read_sql('select * from vaccine', engine)

,id,Date,State,Distributed,Dist_per_100k,Administered,Admin_per_100k
0,1,2023-05,NE,5481710.0,283379.0,3822190.0,197590.0
1,2,2023-05,LA,10282120.0,221178.0,6961453.0,149748.0
2,3,2023-05,GA,28727475.0,270569.0,17124791.0,161290.0
3,4,2023-05,WY,1281755.0,221466.0,854132.0,147580.0
4,5,2023-05,CO,17769135.0,308560.0,13033446.0,226325.0
...,...,...,...,...,...,...,...
61,62,2023-05,AK,2176065.0,297462.0,1328221.0,181564.0
62,63,2023-05,MA,23747420.0,344540.0,18179748.0,263761.0
63,64,2023-05,NM,6176205.0,294550.0,4750638.0,226563.0
64,65,2022-02,RP,43990.0,204301.0,44526.0,206790.0


In [33]:
pd.read_sql('select * from infection', engine)

,id,Date,State,Total_cases,Total_death,Total_recovered,Death_percent,Recovery_percent
0,1,2023-05,AK,297588.0,1468.0,296120.0,0.49,99.51
1,2,2023-05,AL,1659936.0,21138.0,1638798.0,1.27,98.73
2,3,2023-05,AR,995043.0,13115.0,981928.0,1.32,98.68
3,4,2023-05,AS,8331.0,34.0,8297.0,0.41,99.59
4,5,2023-05,AZ,2474154.0,33451.0,2440703.0,1.35,98.65
5,6,2023-05,CA,12251820.0,101886.0,12149934.0,0.83,99.17
6,7,2023-05,CO,1783663.0,14385.0,1769278.0,0.81,99.19
7,8,2023-05,CT,982335.0,12337.0,969998.0,1.26,98.74
8,9,2023-05,DC,178904.0,1435.0,177469.0,0.80,99.20
9,10,2023-05,DE,333772.0,3394.0,330378.0,1.02,98.98


In [27]:
# make an API call to make sure Flask app is working 
import requests
url="http://127.0.0.1:5000/api/v1.0/monthly"
resp=requests.get(url)
resp

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/v1.0/monthly (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001F70B998F98>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))

In [ ]:
data = resp.json()
data